In [1]:
import pandas as pd
import random
import json
import re
import os
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, classification_report
from sklearn.model_selection import GridSearchCV, RandomizedSearchCV
from sklearn.ensemble import BaggingClassifier
from sklearn.multiclass import OneVsRestClassifier
import numpy as np
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.svm import SVC
from tqdm import tqdm
import string
import scipy
import nltk
nltk.download('stopwords')
nltk.download('punkt')
nltk.download('wordnet')
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer, WordNetLemmatizer
from nltk.tokenize import word_tokenize
stop_words = stopwords.words('english')
stemmer = PorterStemmer()
lemmatizer = WordNetLemmatizer()
import torch
from transformers import BertTokenizer, BertForSequenceClassification
from torch.utils.data import DataLoader
from datasets import load_dataset
import pandas as pd
from sklearn.metrics import f1_score
from IPython.display import HTML, display
from tqdm import tqdm
import spacy
import re
import nltk
nltk.download('stopwords')
nltk.download('punkt')
nltk.download('wordnet')
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer, WordNetLemmatizer
from nltk.tokenize import word_tokenize
from nltk.tokenize.treebank import TreebankWordTokenizer
stop_words = stopwords.words('english')
stemmer = PorterStemmer()
lemmatizer = WordNetLemmatizer()
tokenizer = TreebankWordTokenizer()

[nltk_data] Downloading package stopwords to /home/jimbo/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /home/jimbo/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to /home/jimbo/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package stopwords to /home/jimbo/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /home/jimbo/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to /home/jimbo/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [2]:
data_0 = pd.read_csv('../datasets/fossology-master-corrected.csv')
X_0 = data_0["copyright"]
y_0 = data_0["falsePositive"]
X_0 = X_0.drop_duplicates()
y_0 = y_0[X_0.index]

data_1 = pd.read_csv('../datasets/kubernetes-master-corrected.csv')
X_1 = data_1["copyright"]
y_1 = data_1["falsePositive"]
X_1 = X_1.drop_duplicates()
y_1 = y_1[X_1.index]

data_2 = pd.read_csv('../datasets/tensorflow-master-corrected.csv')
X_2 = data_2["copyright"]
y_2 = data_2["falsePositive"]
X_2 = X_2.drop_duplicates()
y_2 = y_2[X_2.index]

data_3 = pd.read_csv('../datasets/fossology-provided-1-corrected.csv')

X_3 = data_3['copyright']
y_3 = data_3['falsePositive']
X_3 = X_3.drop_duplicates()
y_3 = y_3[X_3.index]

data_4 = pd.read_csv('../datasets/fossology-provided-2.csv')

X_4 = data_4['copyright']
y_4 = data_4['falsePositive']
X_4 = X_4.drop_duplicates()
y_4 = y_4[X_4.index]

data_5 = pd.read_csv('../datasets/feature-extraction-paper.csv')

X_5 = data_5['copyright']
y_5 = data_5['falsePositive']
X_5 = X_5.drop_duplicates()
y_5 = y_5[X_5.index]

X = pd.concat([X_0, X_1, X_2, X_3])
y = pd.concat([y_0, y_1, y_2, y_3])

X = X.reset_index(drop=True)
y = y.reset_index(drop=True)

print('Class 0 Percentage: ', len(y[y == 0]) / len(y))
print('Class 1 Percentage: ', len(y[y == 1]) / len(y))

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, shuffle=True)

Class 0 Percentage:  0.7522737712448323
Class 1 Percentage:  0.24772622875516767


In [3]:
def aggregate_reports(reports, print_aggregates=True):
    import pandas as pd
    import numpy as np
    dfs = []
    for metric in ['precision', 'recall', 'f1-score']:
        scores = []
        for report in reports:
            scores.append([report['0'][metric], report['1'][metric]])
        scores = np.array(scores)
        scores = scores[:, :2]
        mean_scores = np.mean(scores, axis=0)
        mean_scores = [f"{score:.6f}" for score in mean_scores]
        df = pd.DataFrame(scores, columns=['0', '1'])
        df.loc['Mean'] = mean_scores
        df['Metric'] = metric
        dfs.append(df)
    if print_aggregates:
        print("## Precision")
        print(dfs[0].to_markdown())
        print("## Recall")
        print(dfs[1].to_markdown())
        print("## F1-score")
        print(dfs[2].to_markdown())
    else:
        return dfs[0], dfs[1], dfs[2]

def get_missclassified_rows(X, y_true, y_pred, only_this_class = [0, 1], return_index=False):
    if type(y_true) != list:
        y_true = y_true.tolist()
    if type(y_pred) != list:
        y_pred = y_pred.tolist()
    if type(X) != list:
        X = X.tolist()
    missclassified_rows = []
    for i in range(len(y_true)):
        if y_true[i] != y_pred[i] and y_true[i] in only_this_class:
            missclassified_rows.append(i)
    if return_index:
        return [(y_pred[i], i, X[i]) for i in missclassified_rows]
    else:
        return [(y_pred[i], X[i]) for i in missclassified_rows]

In [9]:
# Train to save space in code
def train(svm, vectorizer, threshold, preprocess_function,**kwargs):
    X_train_tfidf = vectorizer.fit_transform(preprocess_function(X_train, **kwargs))
    X_test_tfidf = vectorizer.transform(preprocess_function(X_test, **kwargs))
    X_1_tfidf = vectorizer.transform(preprocess_function(X_1, **kwargs))
    X_2_tfidf = vectorizer.transform(preprocess_function(X_2, **kwargs))
    X_3_tfidf = vectorizer.transform(preprocess_function(X_3, **kwargs))
    preprocessed_X = preprocess_function(X, **kwargs)
    X_tfidf = vectorizer.transform(preprocessed_X)
    svm.fit(X_train_tfidf, y_train)
    if True: #svm.probability:
        y_pred = svm.predict_proba(X_test_tfidf)
        y_pred_1 = svm.predict_proba(X_1_tfidf)
        y_pred_2 = svm.predict_proba(X_2_tfidf)
        y_pred_3 = svm.predict_proba(X_3_tfidf)
        y_pred_4 = svm.predict_proba(X_tfidf)
        if threshold is None:
            y_pred_classification = np.argmax(y_pred, axis=1)
            y_pred_1_classification = np.argmax(y_pred_1, axis=1)
            y_pred_2_classification = np.argmax(y_pred_2, axis=1)
            y_pred_3_classification = np.argmax(y_pred_3, axis=1)
            y_pred_4_classification = np.argmax(y_pred_4, axis=1)
        else:
            y_pred_classification = [np.argmax(y) if max(y) > threshold else 0 for y in y_pred]
            y_pred_1_classification = [np.argmax(y) if max(y) > threshold else 0 for y in y_pred_1]
            y_pred_2_classification = [np.argmax(y) if max(y) > threshold else 0 for y in y_pred_2]
            y_pred_3_classification = [np.argmax(y) if max(y) > threshold else 0 for y in y_pred_3]
            y_pred_4_classification = [np.argmax(y) if max(y) > threshold else 0 for y in y_pred_4]
    else:
        y_pred_classification = svm.predict(X_test_tfidf)
        y_pred_1_classification = svm.predict(X_1_tfidf)
        y_pred_2_classification = svm.predict(X_2_tfidf)
        y_pred_3_classification = svm.predict(X_3_tfidf)
        y_pred_4_classification = svm.predict(X_tfidf)
    report = classification_report(y_test, y_pred_classification, output_dict=True)
    report_1 = classification_report(y_1, y_pred_1_classification, output_dict=True)
    report_2 = classification_report(y_2, y_pred_2_classification, output_dict=True)
    report_3 = classification_report(y_3, y_pred_3_classification, output_dict=True)
    report_4 = classification_report(y, y_pred_4_classification, output_dict=True)
    miss_classified_rows_0 = get_missclassified_rows(preprocessed_X, y, y_pred_4_classification, only_this_class=[0], return_index=True)
    miss_classified_rows_1 = get_missclassified_rows(preprocessed_X, y, y_pred_4_classification, only_this_class=[1], return_index=True)
    #aggregate_reports([report, report_1, report_2, report_3, report_4])
    print('Number of missclassifications in class 0: ', report_4['0']['support'] - round(report_4['0']['recall'] * report_4['0']['support']), 'out of a total sample of: ', report_4['0']['support'], ' - about ', round((1 - report_4['0']['recall']) * 100, 2), '% of the class was missclassified')
    print('Number of missclassifications in class 1: ', report_4['1']['support'] - round(report_4['1']['recall'] * report_4['1']['support']), 'out of a total sample of: ', report_4['1']['support'], ' - about ', round((1 - report_4['1']['recall']) * 100, 2), '% of the class was missclassified')
    return svm, vectorizer, miss_classified_rows_0, miss_classified_rows_1, preprocessed_X

In [49]:
# Convert conll2003 to spacy compatible format
!python -m spacy convert '../NER_datasets/default/conll2003/train.txt' '../NER_datasets/spacy_compatible/conll2003' -c ner -n 10
!python -m spacy convert '../NER_datasets/default/conll2003/valid.txt' '../NER_datasets/spacy_compatible/conll2003' -c ner -n 10
!python -m spacy convert '../NER_datasets/default/conll2003/test.txt' '../NER_datasets/spacy_compatible/conll2003' -c ner -n 10

ℹ Auto-detected token-per-line NER format
ℹ Grouping every 10 sentences into a document.
✔ Generated output file (1499 documents):
../NER_datasets/spacy_compatible/conll2003/train.spacy
ℹ Auto-detected token-per-line NER format
ℹ Grouping every 10 sentences into a document.
✔ Generated output file (347 documents):
../NER_datasets/spacy_compatible/conll2003/valid.spacy
ℹ Auto-detected token-per-line NER format
ℹ Grouping every 10 sentences into a document.
✔ Generated output file (369 documents):
../NER_datasets/spacy_compatible/conll2003/test.spacy


In [ ]:
# Convert the broad twitter dataset to spacy compatible format
#!python -m spacy convert '../NER_datasets/default/broad-twitter-dataset/btc.train' '../NER_datasets/spacy_compatible/broad-twitter-dataset' -c ner -n 10
#!python -m spacy convert '../NER_datasets/default/broad-twitter-dataset/btc.dev' '../NER_datasets/spacy_compatible/broad-twitter-dataset' -c ner -n 10
#!python -m spacy convert '../NER_datasets/default/broad-twitter-dataset/btc.test' '../NER_datasets/spacy_compatible/broad-twitter-dataset' -c ner -n 10

In [11]:
def clean_spacy_files(change, change_name, ignore, input_spacy_path, output_spacy_path):
    def filter_ents(ent):
        return ent.label_ not in ignore
    def map_ents(ent):
        new_type = change_name
        if ent.label_ in change:
            return new_type
        else:
            return ent.label_

    nlp = spacy.load("en_core_web_sm")
    doc_bin = spacy.tokens.DocBin().from_disk(input_spacy_path)
    docs = list(doc_bin.get_docs(nlp.vocab))
    for doc in docs:
        filtered_ents = filter(filter_ents, doc.ents)
        doc.set_ents(filtered_ents)
        mapped_ents = [spacy.tokens.Span(doc, ent.start, ent.end, label=map_ents(ent)) for ent in doc.ents]
        doc.set_ents(mapped_ents)
    new_doc_bin = spacy.tokens.DocBin(docs=docs)
    new_doc_bin.to_disk(output_spacy_path)

In [12]:
clean_spacy_files(['PER', 'ORG'], 'ENT', ['MISC', 'LOC'],
                    '../NER_datasets/spacy_compatible/conll2003/train.spacy',
                    '../NER_datasets/spacy_compatible/conll2003/train-cleaned.spacy')
clean_spacy_files(['PER', 'ORG'], 'ENT', ['MISC', 'LOC'],
                    '../NER_datasets/spacy_compatible/conll2003/valid.spacy',
                    '../NER_datasets/spacy_compatible/conll2003/valid-cleaned.spacy')
clean_spacy_files(['PER', 'ORG'], 'ENT', ['MISC', 'LOC'],
                    '../NER_datasets/spacy_compatible/conll2003/test.spacy',
                    '../NER_datasets/spacy_compatible/conll2003/test-cleaned.spacy')

In [15]:
!python -m spacy debug data '../NER_configs/train-colln2003-scratch.cfg'


============================ Data file validation ============================
✔ Pipeline can be initialized with data
✔ Corpus is loadable

=============================== Training stats ===============================
Language: en
Training pipeline: tok2vec, ner
1499 training docs
347 evaluation docs
✔ No overlap between training and evaluation data
⚠ Low number of examples to train a new pipeline (1499)

============================== Vocab & Vectors ==============================
ℹ 204567 total word(s) in the data (23624 unique)
ℹ No word vectors present in the package

========================== Named Entity Recognition ==========================
ℹ 1 label(s)
0 missing value(s) (tokens with '-' label)
✔ Good amount of examples for all labels
✔ Examples without occurrences available for all labels
✔ No entities consisting of or starting/ending with whitespace
✔ No entities crossing sentence boundaries

================================== Summary ==================================
✔

In [16]:
!python -m spacy train '../NER_configs/train-colln2003-scratch.cfg' --output '../NER_models/train-colln2003-scratch'

ℹ Saving to output directory: ../NER_models/train-colln2003-scratch
ℹ Using CPU

=========================== Initializing pipeline ===========================
[2023-09-13 09:29:33,266] [INFO] Set up nlp object from config
[2023-09-13 09:29:33,276] [INFO] Pipeline: ['tok2vec', 'ner']
[2023-09-13 09:29:33,279] [INFO] Created vocabulary
[2023-09-13 09:29:33,280] [INFO] Finished initializing nlp object
[2023-09-13 09:29:35,492] [INFO] Initialized pipeline components: ['tok2vec', 'ner']
✔ Initialized pipeline

============================= Training pipeline =============================
ℹ Pipeline: ['tok2vec', 'ner']
ℹ Initial learn rate: 0.001
E    #       LOSS TOK2VEC  LOSS NER  ENTS_F  ENTS_P  ENTS_R  SCORE 
---  ------  ------------  --------  ------  ------  ------  ------
  0       0          0.00     96.83    4.05    2.53   10.08    0.04
  0     200         72.97   2953.19   75.49   74.65   76.34    0.75
  0     400         86.37   1139.36   79.74   77.24   82.41    0.80
  0     600 

In [17]:
!python -m spacy evaluate '../NER_models/train-colln2003-scratch/model-best/' '../NER_datasets/spacy_compatible/conll2003/test-cleaned.spacy'

ℹ Using CPU

================================== Results ==================================

TOK     -    
NER P   85.62
NER R   84.08
NER F   84.84
SPEED   31215


=============================== NER (per type) ===============================

          P       R       F
ENT   85.62   84.08   84.84



In [18]:
clean_spacy_files(['PER', 'ORG'], 'ENT', ['MISC', 'LOC'],
                    '../NER_datasets/spacy_compatible/copyright-data/1-750.spacy',
                    '../NER_datasets/spacy_compatible/copyright-data/1-750-cleaned.spacy')

In [22]:
# split the 750 documents into train and test
doc_bin = spacy.tokens.DocBin().from_disk('../NER_datasets/spacy_compatible/conll2003/1-750-cleaned.spacy')
nlp = spacy.load("en_core_web_sm")
docs = list(doc_bin.get_docs(nlp.vocab))
train_docs, test_docs = train_test_split(docs, test_size=0.2, random_state=42, shuffle=True)
train_doc_bin = spacy.tokens.DocBin(docs=train_docs)
test_doc_bin = spacy.tokens.DocBin(docs=test_docs)
train_doc_bin.to_disk('../NER_datasets/spacy_compatible/conll2003/1-750-cleaned-train.spacy')
test_doc_bin.to_disk('../NER_datasets/spacy_compatible/conll2003/1-750-cleaned-valid.spacy')

In [24]:
def preprocess_function(sentences, lower=False, replace_copyright_symbols=False,
                        replace_dates=False, remove_numbers=False, remove_punctuation=False,
                        remove_special_characters=False, remove_whitespaces=False, remove_specials_weird=False,
                        remove_stopwords=False, replace_emails=False,
                        tokenize=False, lemmatize=False, glove=False, remove_endlines=False,
                        replace_copyright_symbol_v2=False, replace_entities=False, spacy_model=None):
    if type(sentences) is not list:
        sentences = sentences.to_list()
    for i in range(len(sentences)):
        sentences[i] = str(sentences[i])
    if replace_entities:
        if spacy_model is None: # give error
            raise ValueError('spacy_model must be provided if replace_entities is True')
        else:
            nlp = spacy_model
        sentences = [nlp(sentence) for sentence in sentences]
        new_sentences = []
        for sentence in sentences:
            new_sentence = sentence.text
            for entity in sentence.ents:
                if entity.label_ == 'ENT':
                    new_sentence = re.sub(re.escape(entity.text), ' ENTITY ', new_sentence)
            new_sentences.append(new_sentence)
        sentences = new_sentences
    if replace_dates:
        if glove:
            sentences = [re.sub(r'\d{4}', ' <DATE> ', sentence) for sentence in sentences]
        else:
            sentences = [re.sub(r'\d{4}', ' DATE ', sentence) for sentence in sentences]
    if remove_numbers:
        sentences = [re.sub(r'\d+', ' ', sentence) for sentence in sentences]
    if replace_copyright_symbols:
        if glove:
            symbol_text = ' <COPYRIGHT SYMBOL> '
        else:
            symbol_text = ' COPYRIGHTSYMBOL '
        sentences = [re.sub(r'©', symbol_text, sentence) for sentence in sentences]
        sentences = [re.sub(r'\(c\)', symbol_text, sentence) for sentence in sentences]
        sentences = [re.sub(r'\(C\)', symbol_text, sentence) for sentence in sentences]
    if replace_copyright_symbol_v2:
        if glove:
            symbol_text = ' <COPYRIGHT SYMBOL> '
        else:
            symbol_text = ' COPYRIGHT SYMBOL '
        sentences = [re.sub(r'©', symbol_text, sentence) for sentence in sentences]
        sentences = [re.sub(r'\(c\)', symbol_text, sentence) for sentence in sentences]
        sentences = [re.sub(r'\(C\)', symbol_text, sentence) for sentence in sentences]
    if replace_emails:
        if glove:
            email_text = ' <EMAIL> '
        else:
            email_text = ' EMAIL '
        sentences = [re.sub("""(?:[a-z0-9!#$%&'*+/=?^_`{|}~-]+(?:\.[a-z0-9!#$%&'*+/=?^_`{|}~-]+)*|"(?:[\x01-\x08\x0b\x0c\x0e-\x1f\x21\x23-\x5b\x5d-\x7f]|\\[\x01-\x09\x0b\x0c\x0e-\x7f])*")@(?:(?:[a-z0-9](?:[a-z0-9-]*[a-z0-9])?\.)+[a-z0-9](?:[a-z0-9-]*[a-z0-9])?|\[(?:(?:(2(5[0-5]|[0-4][0-9])|1[0-9][0-9]|[1-9]?[0-9]))\.){3}(?:(2(5[0-5]|[0-4][0-9])|1[0-9][0-9]|[1-9]?[0-9])|[a-z0-9-]*[a-z0-9]:(?:[\x01-\x08\x0b\x0c\x0e-\x1f\x21-\x5a\x53-\x7f]|\\[\x01-\x09\x0b\x0c\x0e-\x7f])+)\])""", email_text, sentence) for sentence in sentences]
    if tokenize:
        if not glove:
            sentences = [''.join(tokenizer.tokenize(sentence)) for sentence in sentences]
        else:
            sentences = [tokenizer.tokenize(sentence) for sentence in sentences]
    if lemmatize:
        lemmatizer = WordNetLemmatizer()
        if not glove:
            sentences = [''.join([lemmatizer.lemmatize(word) for word in sentence]) for sentence in sentences]
        else:
            sentences = [[lemmatizer.lemmatize(word) for word in sentence] for sentence in sentences]
    if remove_punctuation:
        sentences = [re.sub(r'[^\w\s]', ' ', sentence) for sentence in sentences]
    if remove_special_characters:
        sentences = [re.sub(r'[^a-zA-Z0-9]', ' ', sentence) for sentence in sentences]
    if remove_specials_weird:
        sentences = [re.sub(r'[^a-zA-Z0-9]', '', sentence) for sentence in sentences]
    if lower:
        sentences = [sentence.lower() for sentence in sentences]
    if remove_stopwords:
        sentences = [re.sub(r'\b(?:{})\b'.format('|'.join(stopwords.words('english'))), ' ', sentence)
                     for sentence in sentences]
    if remove_endlines:
        sentences = [re.sub(r'\n', ' ', sentence) for sentence in sentences]
    if remove_whitespaces:
        sentences = [re.sub(r' {2,}', ' ', sentence) for sentence in sentences]
    return sentences

In [25]:
clf = OneVsRestClassifier(SVC(probability=True, C=25))
entities = train(clf, TfidfVectorizer(ngram_range=(1, 2), binary=True), None, preprocess_function,
                                            replace_dates=True, remove_numbers=True, replace_copyright_symbols=True,
                                            remove_whitespaces=True, lower=True, remove_special_characters=True, 
                                            replace_emails=True, replace_entities=True,
                                            spacy_model=spacy.load('../NER_models/train-colln2003-scratch//model-best'))

Number of missclassifications in class 0:  35.0 out of a total sample of:  16377.0  - about  0.21 % of the class was missclassified
Number of missclassifications in class 1:  43.0 out of a total sample of:  5393.0  - about  0.8 % of the class was missclassified


In [27]:
clf = OneVsRestClassifier(SVC(probability=True, C=25))
entities_99 = train(clf, TfidfVectorizer(ngram_range=(1, 2), binary=True), 0.99, preprocess_function,
                                            replace_dates=True, remove_numbers=True, replace_copyright_symbols=True,
                                            remove_whitespaces=True, lower=True, remove_special_characters=True, 
                                            replace_emails=True, replace_entities=True,
                                            spacy_model=spacy.load('../NER_models/train-colln2003-scratch//model-best'))

Number of missclassifications in class 0:  8.0 out of a total sample of:  16377.0  - about  0.05 % of the class was missclassified
Number of missclassifications in class 1:  326.0 out of a total sample of:  5393.0  - about  6.04 % of the class was missclassified


In [28]:
entities_99[2]

[(1, 349, ' entity entity '),
 (1, 2608, 'copyright eberhard mattes'),
 (1, 2852, ' entity '),
 (1, 8186, ' entity '),
 (1,
  11786,
  'copyright copyrightsymbol year copyright holders the entity entity entity '),
 (1, 16928, ' entity entity '),
 (1, 17591, ' entity '),
 (1, 17694, 'copyright copyrightsymbol all rights reserved ')]

In [38]:
X[349]

'Copyright 2015,2020, Siemens AG'

In [37]:
preprocess_function(X[349:351], replace_entities=True, spacy_model=spacy.load('../NER_models/train-colln2003-scratch/model-best'))

[' ENTITY ,  ENTITY ', 'Copyright 2015-2017 Siemens  ENTITY ']

In [39]:
X[2852]

'Copyright Beman Dawes 2000'

In [40]:
!python -m spacy train '../NER_configs/fine-tune-750-copyright-data.cfg' --output '../NER_models/fine-tune-750-copyright-data' 

✔ Created output directory:
../NER_configs/fine-tune-750-copyright-data
ℹ Saving to output directory:
../NER_configs/fine-tune-750-copyright-data
ℹ Using CPU

=========================== Initializing pipeline ===========================
[2023-09-13 10:04:02,837] [INFO] Set up nlp object from config
[2023-09-13 10:04:02,847] [INFO] Pipeline: ['tok2vec', 'ner']
[2023-09-13 10:04:02,850] [INFO] Created vocabulary
[2023-09-13 10:04:02,851] [INFO] Finished initializing nlp object
[2023-09-13 10:04:03,142] [INFO] Initialized pipeline components: ['tok2vec', 'ner']
✔ Initialized pipeline

============================= Training pipeline =============================
ℹ Pipeline: ['tok2vec', 'ner']
ℹ Initial learn rate: 0.001
E    #       LOSS TOK2VEC  LOSS NER  ENTS_F  ENTS_P  ENTS_R  SCORE 
---  ------  ------------  --------  ------  ------  ------  ------
  0       0          0.00     46.33    0.00    0.00    0.00    0.00
  2     200         67.24   1516.72   89.20   92.90   85.79    0.89
  

In [42]:
clf = OneVsRestClassifier(SVC(probability=True, C=25))
entities = train(clf, TfidfVectorizer(ngram_range=(1, 2), binary=True), None, preprocess_function,
                                            replace_dates=True, remove_numbers=True, replace_copyright_symbols=True,
                                            remove_whitespaces=True, lower=True, remove_special_characters=True, 
                                            replace_emails=True, replace_entities=True,
                                            spacy_model=spacy.load('../NER_models/fine-tune-750-copyright-data/model-best'))

Number of missclassifications in class 0:  26.0 out of a total sample of:  16377.0  - about  0.16 % of the class was missclassified
Number of missclassifications in class 1:  34.0 out of a total sample of:  5393.0  - about  0.63 % of the class was missclassified


In [45]:
clf = OneVsRestClassifier(SVC(probability=True, C=25))
entities_finetuned_99 = train(clf, TfidfVectorizer(ngram_range=(1, 2), binary=True), 0.99, preprocess_function,
                                            replace_dates=True, remove_numbers=True, replace_copyright_symbols=True,
                                            remove_whitespaces=True, lower=True, remove_special_characters=True, 
                                            replace_emails=True, replace_entities=True,
                                            spacy_model=spacy.load('../NER_models/fine-tune-750-copyright-data/model-best'))

Number of missclassifications in class 0:  4.0 out of a total sample of:  16377.0  - about  0.02 % of the class was missclassified
Number of missclassifications in class 1:  348.0 out of a total sample of:  5393.0  - about  6.45 % of the class was missclassified


In [46]:
entities_finetuned_99[2]

[(1,
  7437,
  'copyright notice a class top href releaseheader date a h pre class licensetext id licensetext entity grants reproduction and use rights to all parties provided that this comment is maintained in the copy pre li '),
 (1,
  11786,
  'copyright copyrightsymbol year copyright holders the entity upl version '),
 (1,
  13458,
  ' copyrightsymbol the entity date n nthis entity the eupl applies to the entity as defined below which is provided under the terms of this licence any use of the work other than as authorised under this licence is prohibited to the extent such use is covered'),
 (1, 17694, 'copyright copyrightsymbol all rights reserved ')]

In [47]:
X[7437]

'Copyright-Notice<a class="top" href="#releaseHeader">&#8679;</a></h3> pre class="licenseText" id="licenseText377"> Third Eye Software, Inc. grants reproduction and use rights to all parties, PROVIDED that this comment is maintained in the copy. pre> li>'

In [76]:
X[11786]

'Copyright (c) [year] [copyright holders] The Universal Permissive License (UPL), Version 1.0'

In [51]:
# Use the current NER model to label the entire dataset and go through it using doccano once more
# to label the entities that were not labeled correctly

nlp = spacy.load('../NER_models/fine-tune-750-copyright-data/model-best')
X_ner = [nlp(sentence) for sentence in tqdm(X)]

def text_to_doccano(sentences):
    """
    :text (str): source text
    Returns (list (dict)): deccano format json
    """
    djson = list()
    #doc = nlp(text)
    for sentence in tqdm(sentences):
        labels = list()
        for e in sentence.ents:
            labels.append([e.start_char, e.end_char, e.label_])
        djson.append({'text': sentence.text, "labels": labels})
    return djson

doccano_json = text_to_doccano(X_ner)
with open('dataset-ner.jsonl', 'w') as f:
    for item in doccano_json:
        f.write(json.dumps(item) + '\n')

100%|██████████| 21770/21770 [00:00<00:00, 76515.56it/s] 


In [53]:
# Convert JSONL to spacy compatible format
import json
import spacy
from spacy.tokens import Doc, DocBin
from spacy.training import Example
from spacy.util import minibatch

def load_data(file_path):
    """
    Load JSONL data from a file and convert it to spaCy's training format.
    :param file_path: The path of the JSONL file.
    :return: A list of training examples.
    """
    data = []
    with open(file_path, 'r') as f:
        for line in f:
            item = json.loads(line)
            text = item['text']
            entities = [(e[0], e[1], e[2]) for e in item['label']]
            data.append((text, {'entities': entities}))
    return data

def save_data(data, file_path):
    """
    Save data in spaCy's binary format.
    :param data: The training data.
    :param file_path: The path of the output file.
    """
    nlp = spacy.blank('en')
    doc_bin = DocBin()
    for text, annotations in data:
        doc = nlp.make_doc(text)
        example = Example.from_dict(doc, annotations)
        doc_bin.add(example.reference)
    doc_bin.to_disk(file_path)

# Load the data
data = load_data('../NER_datasets/default/copyright-data/1-1523.jsonl')

# Save the data
save_data(data, '../NER_datasets/spacy_compatible/copyright-data/1-1523.spacy')


/home/jimbo/.local/lib/python3.8/site-packages/spacy/training/iob_utils.py:149: UserWarning: [W030] Some entities could not be aligned in the text "Copyright (C) 2005 SUSE Linux Products GmbH. Macie..." with entities "[(20, 43, 'ENT'), (45, 60, 'ENT'), (89, 103, 'ENT'...". Use `spacy.training.offsets_to_biluo_tags(nlp.make_doc(text), entities)` to check the alignment. Misaligned entities ('-') will be ignored during training.
  warnings.warn(
/home/jimbo/.local/lib/python3.8/site-packages/spacy/training/iob_utils.py:149: UserWarning: [W030] Some entities could not be aligned in the text "Copyright (c) 2005 Red Hat, Inc." with entities "[(19, 31, 'ENT')]". Use `spacy.training.offsets_to_biluo_tags(nlp.make_doc(text), entities)` to check the alignment. Misaligned entities ('-') will be ignored during training.
  warnings.warn(
/home/jimbo/.local/lib/python3.8/site-packages/spacy/training/iob_utils.py:149: UserWarning: [W030] Some entities could not be aligned in the text "Copyright 2001,

In [54]:
# split the 1523 documents into train and test - 80, 20 split.
doc_bin = spacy.tokens.DocBin().from_disk('../NER_datasets/spacy_compatible/copyright-data/1-1523.spacy')
nlp = spacy.load("en_core_web_sm")
docs = list(doc_bin.get_docs(nlp.vocab))
train_docs, test_docs = train_test_split(docs, test_size=0.2, random_state=42, shuffle=True)
train_doc_bin = spacy.tokens.DocBin(docs=train_docs)
test_doc_bin = spacy.tokens.DocBin(docs=test_docs)
train_doc_bin.to_disk('../NER_datasets/spacy_compatible/copyright-data/1-1523-train.spacy')
test_doc_bin.to_disk('../NER_datasets/spacy_compatible/copyright-data/1-1523-valid.spacy')

### Fine tune once more on the 1523 

In [56]:
!python -m spacy train '../NER_configs/fine-tune-1523-copyright-data.cfg' --output '../NER_models/fine-tune-1523-copyright-data' 

✔ Created output directory:
../NER_models/fine-tune-1523-copyright-data
ℹ Saving to output directory:
../NER_models/fine-tune-1523-copyright-data
ℹ Using CPU

=========================== Initializing pipeline ===========================
[2023-09-13 12:46:14,094] [INFO] Set up nlp object from config
[2023-09-13 12:46:14,104] [INFO] Pipeline: ['tok2vec', 'ner']
[2023-09-13 12:46:14,107] [INFO] Created vocabulary
[2023-09-13 12:46:14,108] [INFO] Finished initializing nlp object
[2023-09-13 12:46:14,672] [INFO] Initialized pipeline components: ['tok2vec', 'ner']
✔ Initialized pipeline

============================= Training pipeline =============================
ℹ Pipeline: ['tok2vec', 'ner']
ℹ Initial learn rate: 0.001
E    #       LOSS TOK2VEC  LOSS NER  ENTS_F  ENTS_P  ENTS_R  SCORE 
---  ------  ------------  --------  ------  ------  ------  ------
  0       0          0.00     40.00    0.00    0.00    0.00    0.00
  1     200         74.16   1448.48   92.49   92.25   92.74    0.92
  

### Testing out the model after on the feature extraction paper dataset

In [62]:
nlp = spacy.load('../NER_models/fine-tune-1523-copyright-data/model-best')
processed_X_5_ents = [nlp(sentence) for sentence in tqdm(X_5)]

100%|██████████| 2081/2081 [00:04<00:00, 430.29it/s]


In [64]:
spacy.displacy.serve(processed_X_5_ents, style="ent", auto_select_port=True)


Using the 'ent' visualizer
Serving on http://0.0.0.0:5000 ...

Shutting down server on port 5000.


### employ semi-supervised learning to train on the entire dataset

In [65]:
nlp = spacy.load('../NER_models/fine-tune-1523-copyright-data/model-best')
processed_X_ents = [nlp(sentence) for sentence in tqdm(X)]

100%|██████████| 21770/21770 [00:52<00:00, 412.29it/s]


In [67]:
train_docs, valid_docs = train_test_split(processed_X_ents, test_size=0.2, random_state=42, shuffle=True)
train_doc_bin = spacy.tokens.DocBin(docs=train_docs)
valid_doc_bin = spacy.tokens.DocBin(docs=valid_docs)
train_doc_bin.to_disk('../NER_datasets/spacy_compatible/copyright-data/enitre-dataset-semi-supervised-train.spacy')
valid_doc_bin.to_disk('../NER_datasets/spacy_compatible/copyright-data/enitre-dataset-semi-supervised-valid.spacy')

In [68]:
!python -m spacy train '../NER_configs/train-semi-supervised-dataset-1.cfg' --output '../NER_models/train-semi-supervised-dataset-1' 

✔ Created output directory:
../NER_models/train-semi-supervised-dataset-1
ℹ Saving to output directory:
../NER_models/train-semi-supervised-dataset-1
ℹ Using CPU

=========================== Initializing pipeline ===========================
[2023-09-13 13:01:52,765] [INFO] Set up nlp object from config
[2023-09-13 13:01:52,775] [INFO] Pipeline: ['tok2vec', 'ner']
[2023-09-13 13:01:52,779] [INFO] Created vocabulary
[2023-09-13 13:01:52,780] [INFO] Finished initializing nlp object
[2023-09-13 13:02:00,061] [INFO] Initialized pipeline components: ['tok2vec', 'ner']
✔ Initialized pipeline

============================= Training pipeline =============================
ℹ Pipeline: ['tok2vec', 'ner']
ℹ Initial learn rate: 0.001
E    #       LOSS TOK2VEC  LOSS NER  ENTS_F  ENTS_P  ENTS_R  SCORE 
---  ------  ------------  --------  ------  ------  ------  ------
  0       0          0.00     32.67    1.84    4.13    1.19    0.02
  0     200        207.89   1674.93   88.88   92.55   85.50    0.8

In [70]:
nlp = spacy.load('../NER_models/train-semi-supervised-dataset-1/model-best')
processed_X_5_ents = [nlp(sentence) for sentence in tqdm(X_5)]
spacy.displacy.serve(processed_X_5_ents, style="ent", auto_select_port=True)

100%|██████████| 2081/2081 [00:04<00:00, 452.55it/s]
/home/jimbo/.local/lib/python3.8/site-packages/spacy/displacy/__init__.py:108: UserWarning: [W011] It looks like you're calling displacy.serve from within a Jupyter notebook or a similar environment. This likely means you're already running a local web server, so there's no need to make displaCy start another one. Instead, you should be able to replace displacy.serve with displacy.render to show the visualization.
  warnings.warn(Warnings.W011)
/home/jimbo/.local/lib/python3.8/site-packages/spacy/displacy/__init__.py:211: UserWarning: [W006] No entities to visualize found in Doc object. If this is surprising to you, make sure the Doc was processed using a model that supports named entity recognition, and check the `doc.ents` property manually if necessary.
  warnings.warn(Warnings.W006)



Using the 'ent' visualizer
Serving on http://0.0.0.0:5000 ...

Shutting down server on port 5000.


In [72]:
clf = OneVsRestClassifier(SVC(probability=True, C=25))
entities_all_data_semi_supervised = train(clf, TfidfVectorizer(ngram_range=(1, 2), binary=True), None, preprocess_function,
                                            replace_dates=True, remove_numbers=True, replace_copyright_symbols=True,
                                            remove_whitespaces=True, lower=True, remove_special_characters=True, 
                                            replace_emails=True, replace_entities=True,
                                            spacy_model=spacy.load('../NER_models/train-semi-supervised-dataset-1/model-best'))

Number of missclassifications in class 0:  23.0 out of a total sample of:  16377.0  - about  0.14 % of the class was missclassified
Number of missclassifications in class 1:  44.0 out of a total sample of:  5393.0  - about  0.82 % of the class was missclassified


In [73]:
clf = OneVsRestClassifier(SVC(probability=True, C=25))
entities_all_data_semi_supervised_99 = train(clf, TfidfVectorizer(ngram_range=(1, 2), binary=True), 0.99, preprocess_function,
                                            replace_dates=True, remove_numbers=True, replace_copyright_symbols=True,
                                            remove_whitespaces=True, lower=True, remove_special_characters=True, 
                                            replace_emails=True, replace_entities=True,
                                            spacy_model=spacy.load('../NER_models/train-semi-supervised-dataset-1/model-best'))

Number of missclassifications in class 0:  3.0 out of a total sample of:  16377.0  - about  0.02 % of the class was missclassified
Number of missclassifications in class 1:  294.0 out of a total sample of:  5393.0  - about  5.45 % of the class was missclassified


In [74]:
entities_all_data_semi_supervised_99[2]

[(1,
  11786,
  'copyright copyrightsymbol year copyright holders the entity upl version '),
 (1,
  13458,
  ' copyrightsymbol the entity date n nthis entity the eupl applies to the work or entity as defined below which is provided under the terms of this licence any use of the work other than as authorised under this licence is prohibited to the extent such use is covered'),
 (1, 17694, 'copyright copyrightsymbol all rights reserved ')]

In [75]:
X[13458]

'(c) the European Community 2007\\n\\nThis European Union Public Licence (the \\"EUPL\\") applies to the Work or Software (as defined below) which is provided under the terms of this Licence. Any use of the Work, other than as authorised under this Licence is prohibited (to the extent such use is covered'